In [112]:
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

import iris
from iris.analysis import MEAN
import pandas as pd
from tqdm import tqdm

In [2]:
gws = Path("/gws/pw/j07/workshop/users/ukcgfi-hackathon/")

depresys = gws / "data/decadal/depresys/dcppA-hindcast_backup/"
depresys_extra = (
    "s{start_year:04d}-r{ensemble_member:d}i1p1f2/Amon/"
    "ts_Amon_HadGEM3-GC31-MM_dcppA-hindcast_"
    "s{start_year:04d}-r{ensemble_member:d}i1p1f2_gn_{year:04d}01-{year:04d}12.nc"
)
depresys_glob = "s*-r*i1p1f2/Amon/ts_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s*-r*_gn_*01-*12.nc"

In [3]:
all_files = sorted(list(depresys.rglob(depresys_glob)))

In [65]:
# (5N-5S, 170W-120W)
enso_region = iris.Constraint(latitude=lambda y: -5 <= y <=5, longitude=lambda x: 190 <= x <= 240)

climatology = iris.load_cube(depresys / "../climatology_files/climatology_1960-2010.nc")
climatology = climatology.collapsed("realization", iris.analysis.MEAN)
for coord in ["latitude", "longitude"]:
    climatology.coord(coord).guess_bounds()

climatology = climatology.extract(enso_region)
weights = iris.analysis.cartography.area_weights(climatology)

/home/users/train187/miniforge3/envs/core/lib/python3.13/site-packages/iris/coords.py:2172: IrisVagueMetadataWarning: Cannot check if coordinate is contiguous: Invalid operation for 'realization', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'realization'. Ignoring bounds.
  warnings.warn(


In [56]:
t = cubes.concatenate_cube().extract(enso_region)

In [67]:
t.extract(enso_region).coord("latitude").points

array([-5.277771  , -4.72221375, -4.16666412, -3.61110687, -3.05554962,
       -2.49999237, -1.94444275, -1.3888855 , -0.83332825, -0.277771  ,
        0.27778625,  0.83333588,  1.38889313,  1.94445038,  2.50000763,
        3.05555725,  3.6111145 ,  4.16667175,  4.722229  ])

In [66]:
climatology.coord("latitude").points

array([-5.277771  , -4.7222137 , -4.166664  , -3.6111069 , -3.0555496 ,
       -2.4999924 , -1.9444427 , -1.3888855 , -0.83332825, -0.277771  ,
        0.27778625,  0.8333359 ,  1.3888931 ,  1.9444504 ,  2.5000076 ,
        3.0555573 ,  3.6111145 ,  4.1666718 ,  4.722229  ], dtype=float32)

In [87]:
#enso_3p4 = []

for start_year in tqdm(range(1960, 2022+1)):
    for ensemble_member in range(1, 10+1):
        filename = depresys / depresys_extra.format(
            start_year=start_year,
            ensemble_member=ensemble_member,
            year=9999,
        ).replace("999901-999912", "*")
        cubes = iris.load(str(filename))
        iris.util.equalise_attributes(cubes)
        t = cubes.concatenate_cube().extract(enso_region) - climatology.data
        t = t.collapsed(["latitude", "longitude"], MEAN, weights=weights)
        t.add_aux_coord(iris.coords.AuxCoord(start_year, long_name="start_year"))
        t.add_aux_coord(iris.coords.AuxCoord(ensemble_member, long_name="ensemble_member"))

        enso_3p4.append(t)

        

 29%|██▊       | 18/63 [05:13<13:03, 17.42s/it]


OSError: [Errno -51] NetCDF: Unknown file format: '/gws/pw/j07/workshop/users/ukcgfi-hackathon/data/decadal/depresys/dcppA-hindcast_backup/s1978-r7i1p1f2/Amon/ts_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s1978-r7i1p1f2_gn_198201-198212.nc'

In [91]:
for start_year in tqdm(range(start_year, start_year+1)):
    for ensemble_member in range(ensemble_member, 10+1):
        filename = depresys / depresys_extra.format(
            start_year=start_year,
            ensemble_member=ensemble_member,
            year=9999,
        ).replace("999901-999912", "*")
        cubes = iris.load(str(filename))
        iris.util.equalise_attributes(cubes)
        t = cubes.concatenate_cube().extract(enso_region) - climatology.data
        t = t.collapsed(["latitude", "longitude"], MEAN, weights=weights)
        t.add_aux_coord(iris.coords.AuxCoord(start_year, long_name="start_year"))
        t.add_aux_coord(iris.coords.AuxCoord(ensemble_member, long_name="ensemble_member"))

        enso_3p4.append(t)

  0%|          | 0/1 [00:00<?, ?it/s]


OSError: One or more of the files specified did not exist:
    * "/gws/pw/j07/workshop/users/ukcgfi-hackathon/data/decadal/depresys/dcppA-hindcast_backup/s2019-r1i1p1f2/Amon/ts_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s2019-r1i1p1f2_gn_*.nc" didn't match any files

In [89]:
for start_year in tqdm(range(start_year+1, 2022)):
    for ensemble_member in range(1, 10+1):
        filename = depresys / depresys_extra.format(
            start_year=start_year,
            ensemble_member=ensemble_member,
            year=9999,
        ).replace("999901-999912", "*")
        cubes = iris.load(str(filename))
        iris.util.equalise_attributes(cubes)
        t = cubes.concatenate_cube().extract(enso_region) - climatology.data
        t = t.collapsed(["latitude", "longitude"], MEAN, weights=weights)
        t.add_aux_coord(iris.coords.AuxCoord(start_year, long_name="start_year"))
        t.add_aux_coord(iris.coords.AuxCoord(ensemble_member, long_name="ensemble_member"))

        enso_3p4.append(t)

 93%|█████████▎| 40/43 [08:06<00:36, 12.17s/it]


OSError: One or more of the files specified did not exist:
    * "/gws/pw/j07/workshop/users/ukcgfi-hackathon/data/decadal/depresys/dcppA-hindcast_backup/s2019-r1i1p1f2/Amon/ts_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s2019-r1i1p1f2_gn_*.nc" didn't match any files

In [96]:
print(enso_3p4[0])

unknown / (K)                       (time: 125)
    Dimension coordinates:
        time                             x
    Scalar coordinates:
        ensemble_member             1
        latitude                    -0.27777099609375 degrees, bound=(-5.555549621582031, 5.000007629394531) degrees
        longitude                   215.0 degrees, bound=(189.16666412353516, 240.83333587646484) degrees
        start_year                  1960
    Cell methods:
        0                           latitude: longitude: mean
    Attributes:
        Conventions                 'CF-1.7 CMIP-6.2'
        activity_id                 'DCPP'
        branch_method               'no parent'
        branch_time_in_child        np.float64(0.0)
        branch_time_in_parent       np.float64(0.0)
        cmor_version                '3.4.0'
        comment                     'Temperature of the lower boundary of the atmosphere'
        cv_version                  '6.2.37.5'
        data_specs_version    

In [117]:
csv_output = []
for cube in tqdm(enso_3p4):
    for subcube in cube.slices_over("time"):
        t = subcube.coord("time")
        t = t.units.num2date(t.points[0])

        dict_out = {coord: [subcube.coord(coord).points[0]] for coord in ["start_year", "ensemble_member"]}
        dict_out["year"] = [t.year]
        dict_out["month"] = [t.month]
        dict_out["Nino3p4"] = [subcube.data[()]]
        csv_output.append(dict_out)

csv_output = pd.concat(csv_output)

 87%|████████▋ | 514/590 [35:19<05:13,  4.12s/it]


OSError: [Errno -51] NetCDF: Unknown file format: '/gws/pw/j07/workshop/users/ukcgfi-hackathon/data/decadal/depresys/dcppA-hindcast_backup/s2011-r5i1p1f2/Amon/ts_Amon_HadGEM3-GC31-MM_dcppA-hindcast_s2011-r5i1p1f2_gn_201601-201612.nc'

In [122]:
csv_half = pd.DataFrame(csv_output)

In [125]:
csv_half["start_year"] = [x[0] for x in csv_half["start_year"]]

In [128]:
for var in ["Nino3p4"]:
    csv_half[var] = [x[0] for x in csv_half[var]]

In [130]:
csv_half.to_csv("../depresys_nino3p4.csv")